# 规则时间序列与不规则时间序列

**时间序列**

时间序列，也称动态序列，是一组按照时间发生先后顺序进行排列的数据点序列。时间序列中的变量通常是在相等间隔的时间段内，依照给定的采样率对某种潜在过程进行观测的结果，这组数据点反映了某个或者某些随机变量随时间不断变化的趋势。时间序列通常用于连续序列的预测问题，其预测的方法包括朴素预测法、简单平均法、移动平均法、加权移动平均、简单指数平滑法、霍尔特线性趋势法、Holt-Winters方法和ARIMA等。

进一步来说，时间序列数据本质上体现的是一个或多个统计指标随时间变化的情况。这些指标数值可以按照年、季度、月、日，甚至实时（秒）进行统计。时间序列分析就是通过分析时间序列数据，来研究统计指标的动态特征和周期特征及其相互关系。此外，时间序列数据的分析还可以应用于异常检测。

综上，时间序列和时间序列数据是对某一或多个现象的动态变化的定量描述，其分析和预测在许多领域都具有重要的应用价值，如经济、商业、社会问题研究等。

**规则时间序列**

规则时间序列，是在时间序列预测问题中，对数据的模式或者趋势做出一些特定的假设。常见的规则有时间序列的均值和方差保持不变，或者时间序列是线性的，自回归模型等。这些规则有助于简化问题的复杂性，使得预测模型更易于构建和理解。在实际应用中，这些规则可能并不总是成立，因此在使用这些规则进行预测时，需要结合实际数据的特性进行调整。

**不规则时间序列**

不规则时间序列，通常指的是数据点没有按照固定的间隔进行采样的时间序列。这意味着在这类时间序列中，数据点的采样时间和采样频率可能会变化，导致时间序列的不规则性。例如，由于某些原因，一个时间段从1993-2013年的地下水水位序列可能出现缺失的数据。

针对这种不规则性，处理方式通常有两种：一种是确定固定间隔，将没有数据的时间点视为丢失的数据，然后通过一些函数来估算这些缺失的数据，例如一些循环神经网络可以通过考虑顺序依赖性来有效地插补序列数据；另一种是直接对不规则时间序列进行建模。

# 常见的时间序列分析技术

使用yfinance库从Python脚本中获取股市数据。

In [15]:
import yfinance as yf

ticker = 'TSLA'
tkr = yf.Ticker(ticker)
df = tkr.history(period = '5d')

df是一个pandas数据框显示的数据。

In [16]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-01-02 00:00:00-05:00,250.080002,251.250000,244.410004,248.419998,104654200,0.0,0.0
2024-01-03 00:00:00-05:00,244.979996,245.679993,236.320007,238.449997,121082600,0.0,0.0
2024-01-04 00:00:00-05:00,239.250000,242.699997,237.729996,237.929993,102629300,0.0,0.0
2024-01-05 00:00:00-05:00,236.860001,240.119995,234.899994,237.490005,92379400,0.0,0.0
2024-01-08 00:00:00-05:00,236.139999,241.250000,235.300003,240.449997,85021800,0.0,0.0


可以看到，数据框是按照Date日期来进行索引的，这意味着数据是一个按时间顺序排列的时间序列。

现在只需要使用Close列，将其输出为pandas序列：

In [17]:
print(df['Close'])

Date
2024-01-02 00:00:00-05:00    248.419998
2024-01-03 00:00:00-05:00    238.449997
2024-01-04 00:00:00-05:00    237.929993
2024-01-05 00:00:00-05:00    237.490005
2024-01-08 00:00:00-05:00    240.449997
Name: Close, dtype: float64


现在采用两种常见的技术：

1. 计算随时间变化的百分比
2. 再滚动时间窗口执行聚合计算

**计算百分比变化**

时间序列分析技术可以用于跟踪观察到的数据随时间变化的程度。

In [18]:
import pandas as pd

print(pd.concat([df['Close'], df['Close'].shift(2)], axis = 1, keys = ['Close', '2DaysShift']))

                                Close  2DaysShift
Date                                             
2024-01-02 00:00:00-05:00  248.419998         NaN
2024-01-03 00:00:00-05:00  238.449997         NaN
2024-01-04 00:00:00-05:00  237.929993  248.419998
2024-01-05 00:00:00-05:00  237.490005  238.449997
2024-01-08 00:00:00-05:00  240.449997  237.929993


可以看到，2DaysShift列的值是Close列移位两天的值，所以存在着NaN的情况。

In [19]:
(df['Close'] - df['Close'].shift(2) / df['Close'].shift(2))

Date
2024-01-02 00:00:00-05:00           NaN
2024-01-03 00:00:00-05:00           NaN
2024-01-04 00:00:00-05:00    236.929993
2024-01-05 00:00:00-05:00    236.490005
2024-01-08 00:00:00-05:00    239.449997
Name: Close, dtype: float64

但是在财务中，需要取对数：

In [20]:
import numpy as np

df['2daysRise'] = np.log(df['Close']) / df['Close'].shift(2)

In [21]:
df['2daysRise']

Date
2024-01-02 00:00:00-05:00         NaN
2024-01-03 00:00:00-05:00         NaN
2024-01-04 00:00:00-05:00    0.022027
2024-01-05 00:00:00-05:00    0.022940
2024-01-08 00:00:00-05:00    0.023043
Name: 2daysRise, dtype: float64

获得一天的收盘价，并且将其除以两个交易日前的收盘价，然后使用NumPy的log()函数取得结果的自然对数。

In [22]:
print(df[['Close', '2daysRise']])

                                Close  2daysRise
Date                                            
2024-01-02 00:00:00-05:00  248.419998        NaN
2024-01-03 00:00:00-05:00  238.449997        NaN
2024-01-04 00:00:00-05:00  237.929993   0.022027
2024-01-05 00:00:00-05:00  237.490005   0.022940
2024-01-08 00:00:00-05:00  240.449997   0.023043


**滚动窗口计算**

将每个值与n个周期的平均值进行比较，这个被称之为叫做滚动窗口计算：创建一个固定大小的时间窗口，移动或滚动时间窗口，对时间窗口内的值执行聚合计算。

In [27]:
df['2daysAvg'] = df['Close'].shift(1).rolling(2).mean()
print(df[['Close', '2daysAvg']])

                                Close    2daysAvg
Date                                             
2024-01-02 00:00:00-05:00  248.419998         NaN
2024-01-03 00:00:00-05:00  238.449997         NaN
2024-01-04 00:00:00-05:00  237.929993  243.434998
2024-01-05 00:00:00-05:00  237.490005  238.189995
2024-01-08 00:00:00-05:00  240.449997  237.709999


# 多元时间序列

多元时间序列是有多个随着时间变化的变量的时间序列。

In [28]:
import pandas as pd
import yfinance as yf

stocks = pd.DataFrame()
tickers = ['MSFT', 'TSLA', 'GM', 'AAPL', 'ORCL', 'AMZN']

for ticker in tickers:
    tkr = yf.Ticker(ticker)
    hist = tkr.history(period = '5d')
    hist = pd.DataFrame(hist[['Close']].rename(columns = {'Close': ticker}))

    if stocks.empty:
        stocks = hist
    else:
        stocks = stocks.join(hist)

这段代码是用来获取多个股票的历史收盘价数据，并将它们合并到一个DataFrame中。

1. 首先，导入了pandas和yfinance库。pandas是一个用于数据处理和分析的Python库，而yfinance是一个用于从Yahoo Finance获取金融数据的Python库。

2. 然后，创建了一个空的DataFrame，名为stocks。这个DataFrame将用于存储所有股票的历史收盘价数据。

3. 定义了一个包含多个股票代码的列表，名为tickers。这些代码是Yahoo Finance上的股票代码。

4. 使用for循环遍历tickers列表中的每个股票代码。对于每个股票代码，执行以下操作：

   a. 使用yfinance库的Ticker类创建一个对象tkr，该对象表示当前股票代码对应的股票。

   b. 使用tkr对象的history方法获取该股票过去5天的历史数据。这个方法返回一个包含历史数据的字典。

   c. 从历史数据字典中提取收盘价数据，并将其转换为一个新的DataFrame。新DataFrame的列名设置为当前股票代码，行索引为日期。

   d. 如果stocks DataFrame为空（即还没有添加任何股票的数据），则将新创建的DataFrame赋值给stocks。否则，使用join方法将新创建的DataFrame与stocks DataFrame进行合并。这样，stocks DataFrame将包含所有股票的历史收盘价数据。

5. 最后，stocks DataFrame将包含所有股票的历史收盘价数据，每列代表一个股票，行索引为日期。

In [29]:
stocks

,MSFT,TSLA,GM,AAPL,ORCL,AMZN
Date,,,,,,
2024-01-02 00:00:00-05:00,370.869995,248.419998,36.049999,185.639999,104.059998,149.929993
2024-01-03 00:00:00-05:00,370.600006,238.449997,35.270000,184.250000,102.459999,148.470001
2024-01-04 00:00:00-05:00,367.940002,237.929993,35.490002,181.910004,102.589996,144.570007
2024-01-05 00:00:00-05:00,367.750000,237.490005,35.990002,181.179993,102.730003,145.240005
2024-01-08 00:00:00-05:00,374.690002,240.449997,36.700001,185.559998,104.660004,149.100006


如上所示是一个多元时间序列数据，在相同的时间跨度内，不同的列显示了不同的股票的收盘价。

处理多元时间序列类似于处理单元时间序列，唯一不同的是你必须处理每行中的几个变量，所以一般采用循环用于迭代序列的列表。

In [31]:
stocks_to_keep = []

for i in stocks.columns:
    if stocks[stocks[i] / stocks[i].shift(1) < 0.97].empty:
        stocks_to_keep.append(i)
print(stocks_to_keep)

['MSFT', 'GM', 'AAPL', 'ORCL', 'AMZN']


接着是输出stocks数据框，让这个数据框只包含 stocks_to_keep 列表的列：

In [33]:
print(stocks[stocks_to_keep])

                                 MSFT         GM        AAPL        ORCL  \
Date                                                                       
2024-01-02 00:00:00-05:00  370.869995  36.049999  185.639999  104.059998   
2024-01-03 00:00:00-05:00  370.600006  35.270000  184.250000  102.459999   
2024-01-04 00:00:00-05:00  367.940002  35.490002  181.910004  102.589996   
2024-01-05 00:00:00-05:00  367.750000  35.990002  181.179993  102.730003   
2024-01-08 00:00:00-05:00  374.690002  36.700001  185.559998  104.660004   

                                 AMZN  
Date                                   
2024-01-02 00:00:00-05:00  149.929993  
2024-01-03 00:00:00-05:00  148.470001  
2024-01-04 00:00:00-05:00  144.570007  
2024-01-05 00:00:00-05:00  145.240005  
2024-01-08 00:00:00-05:00  149.100006  


代码的运行结果会受到运行时候的日期的影响，因为数据是以运行的时间为节点，获得前5日的股票数据的。

多元时间序列的一项重要的任务是：分析数据集中的不同变量之间的关系。

In [34]:
import yfinance as yf
import numpy as np

ticker = 'TSLA'
tkr = yf.Ticker(ticker)
df = tkr.history(period = '1mo')

In [35]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-12-11 00:00:00-05:00,242.740005,243.440002,237.449997,239.740005,97913900,0.0,0.0
2023-12-12 00:00:00-05:00,238.550003,238.990005,233.869995,237.009995,95328300,0.0,0.0
2023-12-13 00:00:00-05:00,234.190002,240.300003,228.199997,239.289993,146286300,0.0,0.0
2023-12-14 00:00:00-05:00,241.220001,253.880005,240.789993,251.050003,160829200,0.0,0.0
2023-12-15 00:00:00-05:00,251.210007,254.130005,248.300003,253.500000,135720800,0.0,0.0
2023-12-18 00:00:00-05:00,253.779999,258.739990,251.360001,252.080002,116416500,0.0,0.0
2023-12-19 00:00:00-05:00,253.479996,258.339996,253.009995,257.220001,106737400,0.0,0.0
2023-12-20 00:00:00-05:00,256.410004,259.839996,247.000000,247.139999,125097000,0.0,0.0
2023-12-21 00:00:00-05:00,251.899994,254.800003,248.550003,254.500000,109594200,0.0,0.0


这里只使用两个列：Close列和Columne列。

In [36]:
df = df[['Close', 'Volume']].rename(columns = {'Close': 'Price'})

In [37]:
df

,Price,Volume
Date,,
2023-12-11 00:00:00-05:00,239.740005,97913900
2023-12-12 00:00:00-05:00,237.009995,95328300
2023-12-13 00:00:00-05:00,239.289993,146286300
2023-12-14 00:00:00-05:00,251.050003,160829200
2023-12-15 00:00:00-05:00,253.500000,135720800
2023-12-18 00:00:00-05:00,252.080002,116416500
2023-12-19 00:00:00-05:00,257.220001,106737400
2023-12-20 00:00:00-05:00,247.139999,125097000
2023-12-21 00:00:00-05:00,254.500000,109594200


为了确定Price列和Volume列之间是否真的存在着关系，需要计算每列中每天的百分比变化。

In [42]:
df['priceRise'] = np.log(df['Price'] / df['Price'].shift(1))

In [43]:
df['volumeRise'] = np.log(df['Volume'] / df['Volume'].shift(1))

In [44]:
df

,Price,Volume,priceRise,volumeRise
Date,,,,
2023-12-11 00:00:00-05:00,239.740005,97913900,NaN,NaN
2023-12-12 00:00:00-05:00,237.009995,95328300,-0.011453,-0.026762
2023-12-13 00:00:00-05:00,239.289993,146286300,0.009574,0.428239
2023-12-14 00:00:00-05:00,251.050003,160829200,0.047976,0.094777
2023-12-15 00:00:00-05:00,253.500000,135720800,0.009712,-0.169743
2023-12-18 00:00:00-05:00,252.080002,116416500,-0.005617,-0.153426
2023-12-19 00:00:00-05:00,257.220001,106737400,0.020185,-0.086803
2023-12-20 00:00:00-05:00,247.139999,125097000,-0.039977,0.158718
2023-12-21 00:00:00-05:00,254.500000,109594200,0.029346,-0.132305


股票市场分析通常考虑的是趋势预测而不是具体的精度。

如果价格与成交量之间存在着依赖性，则可以预计价格高于平均水平的变化与成交量高于平均水平的变化之间的相关性。

In [46]:
print(df[abs(df['priceRise']) > 0.01])

                                Price     Volume  priceRise  volumeRise
Date                                                                   
2023-12-12 00:00:00-05:00  237.009995   95328300  -0.011453   -0.026762
2023-12-14 00:00:00-05:00  251.050003  160829200   0.047976    0.094777
2023-12-19 00:00:00-05:00  257.220001  106737400   0.020185   -0.086803
2023-12-20 00:00:00-05:00  247.139999  125097000  -0.039977    0.158718
2023-12-21 00:00:00-05:00  254.500000  109594200   0.029346   -0.132305
2023-12-26 00:00:00-05:00  256.609985   86892400   0.015988   -0.070611
2023-12-27 00:00:00-05:00  261.440002  106494400   0.018647    0.203422
2023-12-28 00:00:00-05:00  253.179993  113619900  -0.032104    0.064766
2023-12-29 00:00:00-05:00  248.479996  100615300  -0.018738   -0.121554
2024-01-03 00:00:00-05:00  238.449997  121082600  -0.040961    0.145811
2024-01-08 00:00:00-05:00  240.449997   85021800   0.012387   -0.082996
